# Creating Report For Financial Ministry

## Read Data

In [ ]:
!pip install python-docx
!pip list

In [2]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from docx import Document
import pandas as pd

In [6]:
#Receives the data from an excel file in the format provided by the bank of the week's transactions
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename(title="Select Excel Document", filetypes=[("Excel files", "*.csv")])
try:
    data = pd.read_csv(file_path)
except pd.errors.ParserError:
    data = pd.read_csv(file_path, skiprows=6)
data["Date"] = pd.to_datetime(data["Date"])
data["Amount"] = pd.to_numeric(data["Amount"], errors='coerce')
data = data.dropna(subset=["Amount"])
print(data)

In [8]:
#Makes the calculation of the amounts received, spent, and the total
Income = 0
Expense = 0
Big_Income = {}
Big_Expense = {}
start_date = data["Date"].iloc[0].strftime('%m/%d')
end_date = data["Date"].iloc[-1].strftime('%m/%d')
for index, row in data.iterrows():
    if row["Amount"] > 0:
        Income += row["Amount"]
        if row["Amount"] > 100:
            Big_Income[row["Description"]] = round(row["Amount"],2)
    elif row["Amount"] < 0:
        Expense += row["Amount"]
        if row["Amount"] < -100:
            Big_Expense[row["Description"]] = round(row["Amount"],2)
Income = round(Income, 2)
Expense = round(Expense, 2)
Total = Income + Expense
big_income_str = '\n        '.join([f'{key}: ${value:.2f}' for key, value in Big_Income.items()])
big_expense_str = '\n       '.join([f'{key}: ${value:.2f}' for key, value in Big_Expense.items()])

#Sets the info into a string to be written into a Word document
str_Report = f"""{start_date}-{end_date}

Credits: ${Income:.2f}
    {big_income_str}
Debits: ${Expense:.2f}
    {big_expense_str}
TOTAL: {Total}"""
print(str_Report)

In [9]:
#Writes the string of the report into the chosen Word file
root = tk.Tk()
root.withdraw()
try:
    existing_doc_path = filedialog.askopenfilename(title="Select existing Word document", filetypes=[("Word files", "*.docx")])
    doc = Document(existing_doc_path)
    doc.add_paragraph(str_Report)
    doc.save(existing_doc_path)
except:
    messagebox.showinfo("Information","No File was selected")